In [1]:
import numpy as np
import pandas as pd

In [2]:
def create_rank(df, metric_name):
    df[f"{metric_name}_rank"] = (
        df.groupby(["championship", "year"])[metric_name]
        .rank(method="min", ascending=True)
        .astype(int)
    )

    df[f"{metric_name}_rank_gt_naive"] = df["model_name"] == "naive_1"
    df[f"{metric_name}_rank_gt_naive"] = (
        df.sort_values(["championship", "year", f"{metric_name}_rank"])
        .groupby(["championship", "year"])[f"{metric_name}_rank_gt_naive"]
        .cumsum()
        .astype(int)
    )

    return df

In [3]:
def calculate_metrics(country: str):
    df = pd.read_csv(f"../real_data/results/metrics_{country}.csv")
    metrics = (
        df[(df["year"] > 2019) & (df["year"] < 2025)]
        .groupby(["championship", "model_name", "year"])
        .agg(
            num_scenarios=("num_games", "nunique"),
            bs=("brier_score", "mean"),
            rps=("ranked_probability_score", "mean"),
            ls=("log_score", "mean"),
            interval_score=("interval_score", "mean"),
        )
        .reset_index()
    )

    metrics["interval_score"] = metrics["interval_score"].fillna(np.inf)
    metrics = create_rank(metrics, "bs")
    metrics = create_rank(metrics, "rps")
    metrics = create_rank(metrics, "ls")
    metrics = create_rank(metrics, "interval_score")
    metrics = metrics.sort_values(by=["bs_rank", "year"], ignore_index=True)
    cols = [
        "championship",
        "model_name",
        "year",
        "num_scenarios",
        "bs",
        "rps",
        "ls",
        "interval_score",
        "bs_rank",
        "rps_rank",
        "ls_rank",
        "interval_score_rank",
        "bs_rank_gt_naive",
        "rps_rank_gt_naive",
        "ls_rank_gt_naive",
        "interval_score_rank_gt_naive",
    ]

    return (
        metrics[cols]
        .groupby(["championship", "model_name"])
        .agg(
            num_scenarios=("num_scenarios", "sum"),
            bs=("bs", "mean"),
            rps=("rps", "mean"),
            ls=("ls", "mean"),
            interval_score=("interval_score", "mean"),
        )
        .reset_index()
        .sort_values(by=["bs"], ascending=True, ignore_index=True)
        .head(16)
    )

In [4]:
calculate_metrics("brazil")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,brazil,poisson_7,165,0.629883,0.219853,1.046377,1067.429876
1,brazil,poisson_2,165,0.630853,0.220250,1.047620,1066.477463
2,brazil,poisson_9,165,0.640342,0.224739,1.063539,1092.585908
3,brazil,naive_2,165,0.641428,0.225900,1.062325,1081.316727
4,brazil,poisson_4,165,0.641942,0.225518,1.065162,1087.361160
5,brazil,poisson_6,165,0.651206,0.229289,1.076230,1067.827821
6,brazil,poisson_1,165,0.651341,0.229287,1.076486,1068.107383
7,brazil,bradley_terry_3,165,0.652719,0.230384,1.082105,1080.052641
8,brazil,bradley_terry_4,165,0.653953,0.230745,1.084848,1081.986640
9,brazil,poisson_8,165,0.657833,0.232772,1.089227,1090.263948


In [5]:
calculate_metrics("england")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,england,poisson_4,165,0.581111,0.208012,0.977302,1081.568071
1,england,poisson_9,165,0.581323,0.207388,0.977522,1079.190046
2,england,bradley_terry_3,165,0.582336,0.208748,0.979541,1060.560718
3,england,bradley_terry_4,165,0.582957,0.208916,0.980657,1061.442411
4,england,poisson_8,165,0.590382,0.211305,0.991882,1078.028391
5,england,poisson_3,165,0.590864,0.211874,0.994320,1087.860516
6,england,poisson_2,165,0.593121,0.213092,0.993993,1071.871096
7,england,poisson_7,165,0.596081,0.213024,0.998283,1074.336550
8,england,poisson_1,165,0.602352,0.215040,1.007197,1071.256386
9,england,poisson_6,165,0.608212,0.217883,1.015496,1075.659900


In [6]:
calculate_metrics("france")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,france,poisson_4,157,0.606583,0.217992,1.018575,1068.289548
1,france,poisson_3,157,0.608255,0.218816,1.020052,1067.634792
2,france,poisson_9,157,0.608685,0.218087,1.022482,1068.082130
3,france,poisson_2,157,0.609976,0.219631,1.020657,1060.044308
4,france,poisson_8,157,0.611526,0.218674,1.026266,1065.931369
5,france,bradley_terry_3,157,0.611946,0.220301,1.026733,1076.539557
6,france,poisson_7,157,0.612104,0.219449,1.022407,1060.652948
7,france,bradley_terry_4,157,0.612645,0.220520,1.028354,1077.408600
8,france,poisson_1,157,0.613542,0.219146,1.024561,1058.368289
9,france,poisson_6,157,0.617835,0.220985,1.030028,1061.337053


In [7]:
calculate_metrics("germany")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,germany,poisson_7,145,0.614336,0.212052,1.024122,1039.129719
1,germany,poisson_9,145,0.620257,0.214713,1.033997,1052.275007
2,germany,poisson_1,145,0.621211,0.215046,1.034968,1038.395262
3,germany,poisson_2,145,0.623363,0.216272,1.036169,1041.899563
4,germany,poisson_6,145,0.624187,0.216474,1.038676,1039.967397
5,germany,poisson_8,145,0.625871,0.217720,1.044107,1051.538010
6,germany,poisson_4,145,0.626782,0.216938,1.044016,1057.101058
7,germany,bradley_terry_3,145,0.630170,0.219176,1.049908,1049.947580
8,germany,bradley_terry_4,145,0.630319,0.219260,1.050312,1050.421301
9,germany,poisson_3,145,0.634184,0.221025,1.055937,1058.646681


In [8]:
calculate_metrics("italy")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,italy,poisson_1,165,0.606503,0.205108,1.012732,1062.399998
1,italy,poisson_8,165,0.607180,0.204498,1.015278,1064.894179
2,italy,poisson_9,165,0.608264,0.204305,1.016422,1066.138591
3,italy,poisson_4,165,0.608670,0.204212,1.017521,1065.444254
4,italy,poisson_3,165,0.609274,0.204963,1.019610,1065.575951
5,italy,bradley_terry_4,165,0.610638,0.205948,1.019050,1070.902391
6,italy,bradley_terry_3,165,0.610693,0.205958,1.019048,1070.400559
7,italy,poisson_7,165,0.611165,0.207055,1.018681,1067.097515
8,italy,poisson_6,165,0.612973,0.208058,1.021753,1069.559354
9,italy,poisson_2,165,0.614385,0.208557,1.022308,1064.736253


In [9]:
calculate_metrics("netherlands")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,netherlands,poisson_9,145,0.575958,0.199218,0.969666,1065.383711
1,netherlands,poisson_4,145,0.577243,0.199610,0.971619,1066.922418
2,netherlands,poisson_2,145,0.583070,0.201983,0.979890,1056.975085
3,netherlands,poisson_8,145,0.583986,0.202612,0.981986,1065.192364
4,netherlands,poisson_7,145,0.584945,0.202196,0.983387,1058.657065
5,netherlands,poisson_3,145,0.586240,0.203803,0.985494,1069.194292
6,netherlands,poisson_1,145,0.589293,0.202839,0.989770,1056.950829
7,netherlands,bradley_terry_4,145,0.591204,0.206040,0.991850,1070.253728
8,netherlands,bradley_terry_3,145,0.591261,0.206087,0.991865,1070.249873
9,netherlands,poisson_6,145,0.596152,0.206430,0.999521,1061.421492


In [10]:
calculate_metrics("portugal")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,portugal,poisson_9,145,0.564199,0.191836,0.953298,1052.917291
1,portugal,poisson_4,145,0.564517,0.191735,0.953734,1050.910575
2,portugal,bradley_terry_3,145,0.568569,0.195297,0.960496,1053.415102
3,portugal,bradley_terry_4,145,0.568687,0.195354,0.960781,1054.234624
4,portugal,poisson_3,145,0.570477,0.194660,0.963623,1050.360015
5,portugal,poisson_8,145,0.571846,0.195146,0.965789,1051.946751
6,portugal,poisson_7,145,0.572114,0.196374,0.966132,1051.370368
7,portugal,poisson_2,145,0.572658,0.196622,0.966606,1050.732565
8,portugal,poisson_1,145,0.577488,0.198302,0.974221,1048.582312
9,portugal,poisson_6,145,0.582776,0.200646,0.981726,1053.121554


In [11]:
calculate_metrics("spain")

,championship,model_name,num_scenarios,bs,rps,ls,interval_score
0,spain,poisson_2,165,0.605886,0.209128,1.013028,1049.998972
1,spain,poisson_7,165,0.605971,0.209061,1.013230,1050.702320
2,spain,poisson_4,165,0.607474,0.209195,1.019601,1053.782902
3,spain,poisson_9,165,0.608171,0.209502,1.021067,1055.636717
4,spain,poisson_1,165,0.615526,0.213198,1.027449,1050.025152
5,spain,poisson_3,165,0.616282,0.213542,1.033825,1054.966207
6,spain,bradley_terry_3,165,0.616381,0.213097,1.032450,1060.820844
7,spain,bradley_terry_4,165,0.616744,0.213095,1.033881,1061.536646
8,spain,poisson_8,165,0.617007,0.213598,1.034783,1055.896599
9,spain,poisson_6,165,0.618622,0.214604,1.031526,1053.790140


In [12]:
countries = [
    "brazil",
    "england",
    "france",
    "germany",
    "italy",
    "netherlands",
    "portugal",
    "spain",
]

metrics = pd.concat(
    [calculate_metrics(country) for country in countries], ignore_index=True
)

metrics.groupby("model_name").agg(
    bs=("bs", "mean"),
    rps=("rps", "mean"),
    ls=("ls", "mean"),
    interval_score=("interval_score", "mean"),
).reset_index().sort_values(by="bs", ascending=True, ignore_index=True)

,model_name,bs,rps,ls,interval_score
0,poisson_9,0.600900,0.208724,1.007249,1066.526175
1,poisson_4,0.601790,0.209152,1.008441,1066.422498
2,poisson_7,0.603325,0.209883,1.009078,1058.672045
3,poisson_2,0.604164,0.210692,1.010034,1057.841913
4,bradley_terry_3,0.608009,0.212381,1.017768,1065.248359
5,poisson_8,0.608204,0.212041,1.018665,1065.461451
6,bradley_terry_4,0.608394,0.212485,1.018717,1066.023292
7,poisson_3,0.609307,0.212744,1.020510,1068.170584
8,poisson_1,0.609657,0.212246,1.018423,1056.760701
9,poisson_6,0.613995,0.214296,1.024369,1060.335589


In [13]:
countries = ["brazil", "england", "italy", "spain"]

metrics = pd.concat(
    [calculate_metrics(country) for country in countries], ignore_index=True
)

metrics.groupby("model_name").agg(
    bs=("bs", "mean"),
    rps=("rps", "mean"),
    ls=("ls", "mean"),
    interval_score=("interval_score", "mean"),
).reset_index().sort_values(by="bs", ascending=True, ignore_index=True)

,model_name,bs,rps,ls,interval_score
0,poisson_9,0.609525,0.211484,1.019638,1073.387816
1,poisson_4,0.609799,0.211734,1.019896,1072.039097
2,poisson_7,0.610775,0.212248,1.019143,1064.891565
3,poisson_2,0.611061,0.212757,1.019237,1063.270946
4,bradley_terry_3,0.615532,0.214547,1.028286,1067.958690
5,bradley_terry_4,0.616073,0.214676,1.029609,1068.967022
6,poisson_8,0.618101,0.215543,1.032792,1072.270779
7,poisson_3,0.618825,0.215912,1.034743,1074.882223
8,poisson_1,0.618931,0.215658,1.030966,1062.947230
9,poisson_6,0.622753,0.217458,1.036251,1066.709304
